Код обрабатывает изображения исходного датасета.  
Предполагается, что все изображения изначально в правильной ориентации (исправлено вручную).  
Разделение на трейн и тест в соотношении 80:20.  
Трейн: каждое изображение поворачивает на 90, 180 и 270 градусов, присваивает исходному изображению метку 0, остальным 1, 2, 3, соответственно. Таким образом, количество изображений в тренировочной выборке увеличивается в 4 раза.  
Тест: изображение случайным образом поворачивает на 0, 90, 180, 270 градусов, присваивает соответствующую метка класса. Количество изображений не увеличивается.  
Название файла и метку класса записывает в annotations_file_train.csv и annotations_file_test.csv (колонки file_name, target).  

### Импорты

In [219]:
import os
import shutil
from PIL import Image
import pandas as pd
import random

### Константы

In [220]:
# путь к папке с исходными изображениями
DIR_INIT = 'data\\img_dir'
# путь папки для тренировочной выборки
DIR_TRAIN = 'data\\img_dir_train'
# путь папки для тестовой выборки
DIR_TEST = 'data\\img_dir_test'

### Собственные функции

In [221]:
def data_preparation_train(path_='', df_=None):
    """
    Поворачивает каждый файл в директории на 90, 180, 270 гр. и создает датафрейм с разметкой классов
    Args:
        path_ (str): путь к папке с файлами
        df_ (DataFrame): датафрейм для записи меток класса
    Returns:
        нет
    """

    # Формируем список файлов
    file_list_init = os.listdir(path_)

    # Фиксируем имена исходных файлов, метка класса "0"
    for file_name in file_list_init:
        # Проверяем тип файла, добавляем запись в датафрейм
        if file_name.lower().endswith(('.png', '.jpg', '.gpeg')):
            df_.loc[df_.shape[0]] = [file_name, 0]

    # Повороты по 90 градусов, метки классов "1", "2", "3"
    for angle in [90, 180, 270]:
        for file_name in file_list_init:
            if file_name.lower().endswith(('.png', '.jpg', '.gpeg')):
                # открываем изображение, поворачиваем, сохраняем
                file_name_new = str(angle) + '_' + file_name
                file_path = os.path.join(path_, file_name)
                image = Image.open(file_path)
                image_rot = image.rotate(angle=angle, expand=True)
                image_rot.save(os.path.join(path_, file_name_new))
                image.close()
                # фиксируем имя файла и таргет в датафрейме
                if angle == 90:
                    df_.loc[df_.shape[0]] = [file_name_new, 1]
                if angle == 180:
                    df_.loc[df_.shape[0]] = [file_name_new, 2]
                if angle == 270:
                    df_.loc[df_.shape[0]] = [file_name_new, 3]

In [222]:
def data_preparation_test(path_='', df_=None):
    """
    Поворачивает каждый файл в директории один раз случайным образом на 90, 180 или 270 гр.
    и создает датафрейм с разметкой классов
    Args:
        path_ (str): путь к папке с файлами
        df_ (DataFrame): датафрейм для записи меток класса
    Returns:
        нет
    """
    # Формируем список файлов
    file_list_init = os.listdir(path_)

    for file_name in file_list_init:
        # Определяем рандомный угол поворота
        random_angle = random.randrange(0, 271, 90)
        # Проверяем тип файла
        if file_name.lower().endswith(('.png', '.jpg', '.gpeg')):
            # если угол поворота "0"
            if random_angle == 0:
                # фиксируем имя файла и таргет в датафрейме
                df_.loc[len(df_.index)] = [file_name, 0]
            # если угол поворота отличен от нуля
            else:
                # открываем изображение, поворачиваем, сохраняем файл
                #file_name_new = str(random_angle) + '_' + file_name
                file_path = os.path.join(path_, file_name)
                image = Image.open(file_path)
                image = image.rotate(angle=random_angle, expand=True)
                image.save(os.path.join(path_, file_name))
                image.close()
                # фиксируем имя файла и таргет в датафрейме
                if random_angle == 90:
                    df_.loc[len(df_.index)] = [file_name, 1]
                if random_angle == 180:
                    df_.loc[len(df_.index)] = [file_name, 2]
                if random_angle == 270:
                    df_.loc[len(df_.index)] = [file_name, 3]

### Основной код

In [223]:
# Проверяем существование папок для сохранения ротированых изображений, создаем их, если нет.
for i in [DIR_INIT, DIR_TRAIN, DIR_TEST]:
    if not os.path.exists(i):
        os.makedirs(i)

In [224]:
# Создаем датафреймы для фиксации меток класса
df_annot_train = pd.DataFrame(columns=['file_name', 'target'])
df_annot_test = pd.DataFrame(columns=['file_name', 'target'])
df_annot_train

,file_name,target


In [225]:
# Выделяем тренировочную и тестовую выборки, соотношение 80:20
file_list = [f for f in os.listdir(DIR_INIT)
             if f.lower().endswith(('.png', '.jpg', '.gpeg'))]

random.shuffle(file_list)
train_size = int(len(file_list) * 0.8)
train_files = file_list[:train_size]
test_files = file_list[train_size:]
print(f'Тренировочная выборка: {len(train_files)} \nТестовая выборка: {len(test_files)}')

Тренировочная выборка: 138 
Тестовая выборка: 35


In [226]:
# Копируем файлы трейна и теста в соответствующие папки
for fn in train_files:
    sourse_path = os.path.join(DIR_INIT, fn)
    dest_path = os.path.join(DIR_TRAIN, fn)
    shutil.copy(sourse_path, dest_path)

for fn in test_files:
    sourse_path = os.path.join(DIR_INIT, fn)
    dest_path = os.path.join(DIR_TEST, fn)
    shutil.copy(sourse_path, dest_path)

In [227]:
# Обработка тренировочной выборки
data_preparation_train(path_=DIR_TRAIN, df_=df_annot_train)
print(f'Количество наблюдений в тренировочной выборке: {df_annot_train.shape[0]}',
      '\nКоличество наблюдений по классам:')
df_annot_train.target.value_counts()

Количество наблюдений в тренировочной выборке: 552 
Количество наблюдений по классам:


target
0    138
1    138
2    138
3    138
Name: count, dtype: int64

In [228]:
# Обработка тестовой выборки
data_preparation_test(path_=DIR_TEST, df_=df_annot_test)
print(f'Количество наблюдений в тестовой выборке: {df_annot_test.shape[0]}',
      '\nКоличество наблюдений по классам:')
df_annot_test.target.value_counts()

Количество наблюдений в тестовой выборке: 35 
Количество наблюдений по классам:


target
2    11
3     9
1     8
0     7
Name: count, dtype: int64

In [229]:
# Сохраняем дафреймы в файл
df_annot_train.to_csv('data\\'+'annotations_file_train.csv')
df_annot_test.to_csv('data\\'+'annotations_file_test.csv')